<a href="https://colab.research.google.com/github/gracefacetseng/mrc-image-converter/blob/main/converter%20-%20google%20colab%20ver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image File Converter

This notebook supports the following file types:
*   Input: png, jpg, tiff, mrc
*   Output: png, jpg, tiff

When converting mrc files, it may be useful to visualize the original file in napari if you want the file to save as it is displayed in napari.

In [ ]:
#@title 1. Install and import dependencies
import sys
!{sys.executable} -m pip install pathlib mrcfile Pillow numpy tifffile

from google.colab import files
import os
import mrcfile
from PIL import Image as im
import numpy as np
import tifffile as tif

In [ ]:
#@title 2. Import your file and set conversion parameters:
# import file
filename = files.upload()

for fn in filename.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(filename[fn])))

filetype = os.path.splitext(fn)[1]
print(fn)

# check input filetype
valid_filetypes = ['.png', '.tif', '.tiff', '.mrc', '.jpg', '.jpeg']

if filetype in valid_filetypes:
    pass
else:
    print('File type not accepted.')
    sys.exit()

# specify destination filetype
#@markdown Use the dropdown menu to choose which file type to convert your image to:
to_file = 'png' #@param ['png', 'tiff', 'jpg']

def suffix(to_file):
    if to_file == 'png':
        return '.png'
    elif to_file == 'tiff':
        return '.tiff'
    elif to_file == 'jpg':
        return '.jpg'
    else:
        print('File type not accepted. Please try again.')
        sys.exit()

#@markdown Please name the new file:
input_name = '' #@param {type:'string'}
new_name = input_name + suffix(to_file)

In [59]:
#@title 3. Convert your image to an array:
def img_to_array(fn):
    if filetype == '.mrc':
        return mrcfile.mmap(fn, permissive=True).data
    else:
        return np.asarray(im.open(fn))

array = img_to_array(fn)

#@markdown Check this box if you want to convert your image to greyscale. This is necessary to generate a 2-Dimensional image file.
if_greyscale = False #@param {type:'boolean'}
if if_greyscale:
  array = np.mean(array, axis=2)

### 4. Set contrast limits. Details in the code:

In [4]:
# to find the min and max of the array, if necessary
print(np.min(array))
print(np.max(array))

71
255


In [51]:
# set contrast limits
if filetype == '.mrc':
    contrast_min = array.min() # or a specified minimum pixel value
    contrast_max = array.max() # or a specified maximum pixel value

    ## contrast limits of mrc files must be manually adjusted to appear as they look in napari. in rare cases,
    ## the limits are equal to the min and max of the image. if so, use the same parameters for all other image
    ## file types. napari does not seem to adjust the contrast limits in any substantial way; these values may
    ## be found in metadata associated with the image itself. for now, the only way to find these values is by
    ## trial and error. it may be helpful to find the min and max values using the block above. contrast limits
    ## are often somewhere in the middle of that range.

else:
    contrast_min = array.min()
    contrast_max = array.max()

array = np.where(array <= contrast_min, contrast_min, array)
array = np.where(array >= contrast_max, contrast_max, array)
array = (array-np.min(array))/(np.max(array)-np.min(array))

if to_file == 'tiff':
  pass
else:
  array *= 255/array.max()

### 5. Convert the array to the desired image file and retrieve output

In [60]:
# convert array to new image filetype
def array_to_img(array):
    if to_file == 'tiff':
        return tif.imwrite(new_name, array)
    elif to_file == 'png':
        array = array.astype(np.uint8)
        png = im.fromarray(array)
        return png.save(new_name, Format='PNG')
    else:
        array = array.astype(np.uint8)
        jpg = im.fromarray(array).convert('RGB')
        return jpg.save(new_name, Format='JPG')

array_to_img(array)
files.download(new_name)

print(fn, ' successfully converted to ', to_file,', see file ', new_name, sep='')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

test image 3 - RGBA test (2).jpg successfully converted to png, see file demo 2.png
